# Constants to discover where a statement is executed from

I hate the `__name__ == '__main__'`, it throws people off.  Like Y the F R U doing that.

This notebook creates exports three objects that will tell if:

* If the statement was called from a `__main__` module.
* If the statement was called interactively.
* If the statement was called on an imported object.

In [1]:
    __all__ = 'MAIN', 'INTERACTIVE', 'IMPORTED', 'CLI'

In [2]:
    from inspect import currentframe, getouterframes
    from pathlib import Path
    from abc import ABCMeta, abstractmethod

Get the current frame and retreive the first frame called outside of the defining class.  We are basically counting a set of known frames before the caller is found.

In [3]:
    class state(metaclass=ABCMeta):
        """Base class for the state objects."""
        def _retrieve_frames(self): return [
            object.frame for object in getouterframes(currentframe())]
        
        def _retrieve_caller_frame(self): 
            """Frames in stack
            
            other calls frame calls bool calls 
            retrieve_caller_frame calls retrieve_frames 
            """
            (retrieve_caller, retrieve_frames, __bool__, 
             frame, *other) = self._retrieve_frames()
            return frame
        
        @abstractmethod
        def format(self): raise NotImplemented()
        
        def __bool__(self): 
            frame = self._retrieve_caller_frame()
            return self.format(frame)

## Does `__name__ == '__main__'`?

The most common logic circuit.  It is annoying to type, ugly to read, and must go away.

In [4]:
class main(state):
    """Use MAIN to discover if a statement is invoked as a __main__ program.
    
    replaces
    
        __name__ == '__main__'"""
    def format(self, frame):
        name = frame.f_globals.get('__name__')
        return name == '__main__'
    
MAIN = main()

## Is the module being run from the command line as an application?

In [5]:
class cli(state):
    """Use CLI to discover if a statement is invoked as a __main__ program.
    
    replaces
    
        __name__ == '__main__' and __import__('sys').argv == __file__
    """

    def format(self, frame):
        name = frame.f_globals.get('__name__')
        file = frame.f_globals.get('__file__', None)
        return name == '__main__' and file == __import__('sys').argv[0]
    
CLI = cli()

## Is the code executed by IPython or Jupyter?

In [6]:
class interactive(state):
    """Use INTERACTIVE to discover if a statement is invoked from IPython or Jupyter.
    
    replaces
        
        __name__ == '__main__' and globals().get('__file__', None) == None
    """
    def format(self, frame):
        name = frame.f_globals.get('__name__')
        file = frame.f_globals.get('__file__', None)
        return name == '__main__' and file is None
    
INTERACTIVE = interactive()

## Is the module simply being imported?

In [7]:
class imported(state):
    """Use IMPORTED to discover if a statement is invoked as an imported module.
    
    replaces: 
    
        __name__ != '__main__'
    """
    def format(self, frame):
        name = frame.f_globals.get('__name__')
        return name != '__main__'
    
IMPORTED = imported()

# Developer

In [8]:
    def test_imported():
        assert IMPORTED, "the logic is borked."
        assert not any((MAIN, INTERACTIVE, CLI))

In [9]:
    if INTERACTIVE:
        __import__('pytest').main("-s 2018-07-21-whereami.ipynb".split())

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.5.1, py-1.5.3, pluggy-0.6.0
rootdir: C:\Users\deathbeds\deathbeds.github.io\deathbeds, inifile: tox.ini
plugins: testmon-0.9.12, remotedata-0.2.1, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2, importnb-0.4.1, hypothesis-3.66.1
collected 1 item

2018-07-21-whereami.ipynb .

========================== 1 passed in 0.42 seconds ===========================


## Discussion

What are states are important?  Testing? 